In [182]:
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import Dense, LSTM, Embedding
import pickle

In [93]:
# load text
file = open('nmh_lyrics.txt', 'r')
text = file.read()
file.close()

In [94]:
# clean text
text = text.replace('[Verse ', '[verse')
text = text.replace("'", '')
tokens = text.split()
table = str.maketrans('', '', string.punctuation)
tokens = [w.translate(table) for w in tokens]
tokens = [token.lower() for token in tokens]

In [95]:
# saving tokens
length = 25 + 1
sequences = []
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)

In [99]:
# save sequence
data = '\n'.join(sequences)
file = open('clean_nmh_lyrics.txt', 'w')
file.write(data)
file.close()

In [1]:
# load lyrics
file = open('clean_nmh_lyrics.txt', 'r')
text = file.read()
text = text.replace(',', '')
file.close
lines = text.split('\n')

In [2]:
lines

['verse1 when you were young you were the king of carrot flowers and how you built a tower tumbling through the trees in holy rattlesnakes that',
 'when you were young you were the king of carrot flowers and how you built a tower tumbling through the trees in holy rattlesnakes that fell',
 'you were young you were the king of carrot flowers and how you built a tower tumbling through the trees in holy rattlesnakes that fell all',
 'were young you were the king of carrot flowers and how you built a tower tumbling through the trees in holy rattlesnakes that fell all around',
 'young you were the king of carrot flowers and how you built a tower tumbling through the trees in holy rattlesnakes that fell all around your',
 'you were the king of carrot flowers and how you built a tower tumbling through the trees in holy rattlesnakes that fell all around your feet',
 'were the king of carrot flowers and how you built a tower tumbling through the trees in holy rattlesnakes that fell all around y

In [105]:
# tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1

In [4]:
# set up data
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y)
seq_length = X.shape[1]

NameError: name 'np' is not defined

In [3]:
y

NameError: name 'y' is not defined

In [126]:
# model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [127]:
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
2153/2153 [==============================] - 4s 2ms/step - loss: 6.3981 - acc: 0.0390
Epoch 2/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.8398 - acc: 0.0390
Epoch 3/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6748 - acc: 0.0418
Epoch 4/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6395 - acc: 0.0418
Epoch 5/100
2153/2153 [==============================] - 3s 2ms/step - loss: 5.6346 - acc: 0.0418
Epoch 6/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6318 - acc: 0.0376
Epoch 7/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6262 - acc: 0.0418
Epoch 8/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6240 - acc: 0.0418
Epoch 9/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.6172 - acc: 0.0553
Epoch 10/100
2153/2153 [==============================] - 4s 2ms/step - loss: 5.5860 - acc: 0.0687
Epoch 11/100
2153/2

2153/2153 [==============================] - 4s 2ms/step - loss: 1.8918 - acc: 0.5151
Epoch 84/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.8535 - acc: 0.5300
Epoch 85/100
2153/2153 [==============================] - 4s 2ms/step - loss: 1.8230 - acc: 0.5411
Epoch 86/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.7919 - acc: 0.5467
Epoch 87/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.7905 - acc: 0.5444
Epoch 88/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.7417 - acc: 0.5536
Epoch 89/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.7127 - acc: 0.5662
Epoch 90/100
2153/2153 [==============================] - 3s 2ms/step - loss: 1.6812 - acc: 0.5764
Epoch 91/100
2153/2153 [==============================] - 4s 2ms/step - loss: 1.6578 - acc: 0.5787A: 2s - loss: 1.5698 
Epoch 92/100
2153/2153 [==============================] - 4s 2ms/step - loss: 1.6240 - acc: 0.5936
Ep

In [131]:
# save model
model.save('base_model_from_Machine_Learning_Mastery.h5')
# save the tokenizer
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [157]:
# select a seed text
seed_text = lines[np.random.randint(0,len(lines))]

encoded = tokenizer.texts_to_sequences([seed_text])[0][:25]

In [159]:
encoded = np.reshape(encoded, (1, 25))

In [160]:
yhat = model.predict_classes(encoded, verbose=0)

In [175]:
reverse_dict = dict([(index, word) for word, index in tokenizer.word_index.items()])

In [209]:
def generate_seq(model=model,
                 tokenizer=tokenizer,
                 seq_length=seq_length,
                 seed_text=lines[np.random.randint(0,len(lines))],
                 n_words=500):
    result = []
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        yhat = model.predict_classes(encoded, verbose=0)
        word = reverse_dict[yhat[0]]
        in_text = in_text + ' ' + word
        result.append(word)
    return ' '.join(result)

In [210]:
generate_seq()

'ashes left born and theyll need screams and falls apart chorus two your mouth to say ourselves at least enough to carry to carry on standing communist the mountain love the enemies with jesus enemies i sank take you you will live forever you tomatoes and radio wires and wont safe and retire on a spine to say hang for your hollow ways moving your mouth to pull out to have found and ugly bursting to carry to carry to say again moving a face i when all bridge do do do do do do do do do do do do do do do do do do do do do do i will know you buried you will be one you you her sister and mother and fivehundred families and twist we is around a place all bend and spitting wanting and could left where a dead dog as and sleep to tore to pull out to be myself again things to try but your mouth on your miracles aimed to say keys for two the point shout with know you will love you spitting did she will take off our place you fivehundred wait for the rest of your eyes of some holy enemy ill crush 

In [ ]:
import pandas as pd
import numpy as np

In [212]:
df = pd.read_csv('all_songs.csv')

In [213]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,is_alt_country,is_alt_rock,is_experimental_rock,is_hip_hop,is_pop_songs,is_rock_songs
0,0,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,A Minor Place,1.0,[hook] n I've been to a minor place n And I ca...,https://genius.com/Bonnie-prince-billy-a-minor...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,Nomadic Revery (All Around),2.0,"Today was one where, lost in thought n I reall...",https://genius.com/Bonnie-prince-billy-nomadic...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,I See a Darkness,3.0,Well you're my friend n (That's what you told ...,https://genius.com/Bonnie-prince-billy-i-see-a...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,Another Day Full of Dread,4.0,I like to have a good time n Any of my friends...,https://genius.com/Bonnie-prince-billy-another...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,Death to Everyone,5.0,"I am here, right here n Where God puts non asu...",https://genius.com/Bonnie-prince-billy-death-t...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,Knockturne,6.0,Fire-burned and blew out flowers n Showing me ...,https://genius.com/Bonnie-prince-billy-knocktu...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,Madeleine Mary,7.0,Sing a song of madeleine-mary n A tune that al...,https://genius.com/Bonnie-prince-billy-madelei...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,Song for the New Breed,8.0,Your little feet n Your sharp teeth n The way ...,https://genius.com/Bonnie-prince-billy-song-fo...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,Today I Was an Evil One,9.0,Tonight my eyes were hurting much n As I had s...,https://genius.com/Bonnie-prince-billy-today-i...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,0,Bonnie 'Prince' Billy,I See a Darkness,https://genius.com/albums/Bonnie-Prince-Billy/...,Black,10.0,"Black, you are my enemy n And I cannot get clo...",https://genius.com/Bonnie-prince-billy-black-l...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from io import StringIO